<a href="https://colab.research.google.com/github/autowiki4/Deep-Chem-Tutorials/blob/main/examples/tutorials/Going_Deeper_on_Molecular_Featurizations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Going Deeper On Molecular Featurizations

One of the most important steps of doing machine learning on molecular data is transforming the data into a form amenable to the application of learning algorithms. This process is broadly called "featurization" and involves turning a molecule into a vector or tensor of some sort. There are a number of different ways of doing that, and the choice of featurization is often dependent on the problem at hand. We have already seen two such methods: molecular fingerprints, and `ConvMol` objects for use with graph convolutions. In this tutorial we will look at some of the others.

## Colab

This tutorial and the rest in this sequence can be done in Google colab. If you'd like to open this notebook in colab, you can use the following link.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deepchem/deepchem/blob/master/examples/tutorials/Going_Deeper_on_Molecular_Featurizations.ipynb)



In [1]:
!pip install deepchem
import deepchem
import warnings
warnings.filterwarnings('ignore')
deepchem.__version__

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 36.5 MB/s eta 0:00:00


wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


'2.8.0'

## Featurizers

In DeepChem, a method of featurizing a molecule (or any other sort of input) is defined by a `Featurizer` object.  There are three different ways of using featurizers.

1. When using the MoleculeNet loader functions, you simply pass the name of the featurization method to use.  We have seen examples of this in earlier tutorials, such as `featurizer='ECFP'` or `featurizer='GraphConv'`.

2. You also can create a Featurizer and directly apply it to molecules.  For example:

In [35]:
import deepchem as dc

featurizer = dc.feat.CircularFingerprint()
print(featurizer(['CC', 'CCC', 'CCO']))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


[16:03:15] DEPRECATION WARNING: please use MorganGenerator
[16:03:15] DEPRECATION WARNING: please use MorganGenerator
[16:03:15] DEPRECATION WARNING: please use MorganGenerator


3. When creating a new dataset with the DataLoader framework, you can specify a Featurizer to use for processing the data.  We will see this in a future tutorial.

We use propane (CH<sub>3</sub>CH<sub>2</sub>CH<sub>3</sub>, represented by the SMILES string `'CCC'`) as a running example throughout this tutorial. Many of the featurization methods use conformers of the molecules. A conformer can be generated using the `ConformerGenerator` class in `deepchem.utils.conformers`.

### RDKitDescriptors

`RDKitDescriptors` featurizes a molecule by using RDKit to compute values for a list of descriptors. These are basic physical and chemical properties: molecular weight, polar surface area, numbers of hydrogen bond donors and acceptors, etc. This is most useful for predicting things that depend on these high level properties rather than on detailed molecular structure.

Intrinsic to the featurizer is a set of allowed descriptors, which can be accessed using `RDKitDescriptors.allowedDescriptors`. The featurizer uses the descriptors in `rdkit.Chem.Descriptors.descList`, checks if they are in the list of allowed descriptors, and computes the descriptor value for the molecule.

Let's print the values of the first ten descriptors for propane.

In [36]:
from rdkit.Chem.Descriptors import descList
rdkit_featurizer = dc.feat.RDKitDescriptors()
features = rdkit_featurizer(['CCC'])[0]
descriptors = [i[0] for i in descList]
for feature, descriptor in zip(features[:10], descriptors):
    print(descriptor, feature)

MaxAbsEStateIndex 0.9182958340544894
MaxEStateIndex 1.3523744386698802
MinAbsEStateIndex -1.4760582438091696
MinEStateIndex 1.55881365076144
qed -1.2696136507614397
SPS 3.9177136507614425
MolWt 1.089286349238561
HeavyAtomMolWt 13.425713650761436
ExactMolWt 10.597286349238557
NumValenceElectrons 1.6329931618554523


Of course, there are many more descriptors than this.

In [37]:
print('The number of descriptors present is: ', len(features))

The number of descriptors present is:  217


### WeaveFeaturizer and MolGraphConvFeaturizer

We previously looked at graph convolutions, which use `ConvMolFeaturizer` to convert molecules into `ConvMol` objects.  Graph convolutions are a special case of a large class of architectures that represent molecules as graphs.  They work in similar ways but vary in the details.  For example, they may associate data vectors with the atoms, the bonds connecting them, or both.  They may use a variety of techniques to calculate new data vectors from those in the previous layer, and a variety of techniques to compute molecule level properties at the end.

DeepChem supports lots of different graph based models.  Some of them require molecules to be featurized in slightly different ways.  Because of this, there are two other featurizers called `WeaveFeaturizer` and `MolGraphConvFeaturizer`.  They each convert molecules into a different type of Python object that is used by particular models.  When using any graph based model, just check the documentation to see what featurizer you need to use with it.

### CoulombMatrix

All the models we have looked at so far consider only the intrinsic properties of a molecule: the list of atoms that compose it and the bonds connecting them.  When working with flexible molecules, you may also want to consider the different conformations the molecule can take on.  For example, when a drug molecule binds to a protein, the strength of the binding depends on specific interactions between pairs of atoms.  To predict binding strength, you probably want to consider a variety of possible conformations and use a model that takes them into account when making predictions.

The Coulomb matrix is one popular featurization for molecular conformations.  Recall that the electrostatic Coulomb interaction between two charges is proportional to $q_1 q_2/r$ where $q_1$ and $q_2$ are the charges and $r$ is the distance between them.  For a molecule with $N$ atoms, the Coulomb matrix is a $N \times N$ matrix where each element gives the strength of the electrostatic interaction between two atoms.  It contains information both about the charges on the atoms and the distances between them.  More information on the functional forms used can be found [here](https://journals.aps.org/prl/pdf/10.1103/PhysRevLett.108.058301).

To apply this featurizer, we first need a set of conformations for the molecule.  We can use the `ConformerGenerator` class to do this.  It takes a RDKit molecule, generates a set of energy minimized conformers, and prunes the set to only include ones that are significantly different from each other.  Let's try running it for propane.

In [38]:
from rdkit import Chem

generator = dc.utils.ConformerGenerator(max_conformers=5)
propane_mol = generator.generate_conformers(Chem.MolFromSmiles('CCC'))
print("Number of available conformers for propane: ", len(propane_mol.GetConformers()))

Number of available conformers for propane:  1


It only found a single conformer.  This shouldn't be surprising, since propane is a very small molecule with hardly any flexibility.  Let's try adding another carbon.

In [39]:
butane_mol = generator.generate_conformers(Chem.MolFromSmiles('CCCC'))
print("Number of available conformers for butane: ", len(butane_mol.GetConformers()))

Number of available conformers for butane:  3


Now we can create a Coulomb matrix for our molecule.

In [40]:
coulomb_mat = dc.feat.CoulombMatrix(max_atoms=20)
features = coulomb_mat(propane_mol)
print(features)

[[[36.8581052  12.4868446   7.56196913  2.85804526  2.8594516
    2.85804538  1.46740146  1.46740146  1.14239721  1.14239681
    0.91279492  0.          0.          0.          0.
    0.          0.          0.          0.          0.        ]
  [12.4868446  36.8581052  12.48684444  1.45850729  1.4655121
    1.45850738  2.85689508  2.85689502  1.45850736  1.45850733
    1.46551212  0.          0.          0.          0.
    0.          0.          0.          0.          0.        ]
  [ 7.56196913 12.48684444 36.8581052   1.14239701  0.9127949
    1.14239697  1.46740146  1.46740146  2.85804543  2.85804544
    2.85945197  0.          0.          0.          0.
    0.          0.          0.          0.          0.        ]
  [ 2.85804526  1.45850729  1.14239701  0.5         0.29325366
    0.29200271  0.17113412  0.21092511  0.16800025  0.20540029
    0.13960186  0.          0.          0.          0.
    0.          0.          0.          0.          0.        ]
  [ 2.8594516   1.46551

Notice that many elements are 0.  To combine multiple molecules in a batch we need all the Coulomb matrices to be the same size, even if the molecules have different numbers of atoms.  We specified `max_atoms=20`, so the returned matrix  has size (20, 20).  The molecule only has 11 atoms, so only an 11 by 11 submatrix is nonzero.

### CoulombMatrixEig

An important feature of Coulomb matrices is that they are invariant to molecular rotation and translation, since the interatomic distances and atomic numbers do not change.  Respecting symmetries like this makes learning easier.  Rotating a molecule does not change its physical properties.  If the featurization does change, then the model is forced to learn that rotations are not important, but if the featurization is invariant then the model gets this property automatically.

Coulomb matrices are not invariant under another important symmetry: permutations of the atoms' indices.  A molecule's physical properties do not depend on which atom we call "atom 1", but the Coulomb matrix does.  To deal with this, the `CoulumbMatrixEig` featurizer was introduced, which uses the eigenvalue spectrum of the Coulumb matrix and is invariant to random permutations of the atom's indices.  The disadvantage of this featurization is that it contains much less information ($N$ eigenvalues instead of an $N \times N$ matrix), so models will be more limited in what they can learn.

`CoulombMatrixEig` inherits from `CoulombMatrix` and featurizes a molecule by first computing the Coulomb matrices for different conformers of the molecule and then computing the eigenvalues for each Coulomb matrix. These eigenvalues are then padded to account for variation in number of atoms across molecules.

In [41]:
coulomb_mat_eig = dc.feat.CoulombMatrixEig(max_atoms=20)
features = coulomb_mat_eig(propane_mol)
print(features)

[[60.07620391 29.62963107 22.75497737  0.5713786   0.28781331  0.28548337
   0.27558188  0.18163797  0.17460998  0.17059714  0.166401    0.
   0.          0.          0.          0.          0.          0.
   0.          0.        ]]


### SMILES Tokenization and Numericalization


So far, we have looked at featurization techniques that translate the implicit structural and physical information in SMILES data into more explicit features that help our models learn and make predictions. In this section, we will preprocess SMILES strings into a format that can be fed to sequence models, such as 1D convolutional neural networks and transformers, and enables these models to learn their own representations of molecular properties.

To prepare SMILES strings for a sequence model, we break them down into lists of substrings (called tokens) and turn them into lists of integer values (numericalization). Sequence models use those integer values as indices of an embedding matrix, which contains a vector of floating-point numbers for each token in the vocabulary. These embedding vectors are updated during model training. This process allows the sequence model to learn its own representations of the molecular properties implicit in the training data.

We will use DeepChem's `BasicSmilesTokenizer` and the Tox21 dataset from MoleculeNet to demonstrate the process of tokenizing SMILES.

In [55]:
import numpy as np

In [56]:
tasks, datasets, transformers = dc.molnet.load_tox21(featurizer="Raw")
train_dataset, valid_dataset, test_dataset = datasets
print(train_dataset)

<DiskDataset X.shape: (np.int64(6258),), y.shape: (np.int64(6258), np.int64(12)), w.shape: (np.int64(6258), np.int64(12)), task_names: ['NR-AR' 'NR-AR-LBD' 'NR-AhR' ... 'SR-HSE' 'SR-MMP' 'SR-p53']>


We loaded the datasets with `featurizer="Raw"`. Now we obtain the SMILES from their `ids` attributes.

In [57]:
train_smiles = train_dataset.ids
valid_smiles = valid_dataset.ids
test_smiles = test_dataset.ids
print(train_smiles[:5])

['CC(O)(P(=O)(O)O)P(=O)(O)O' 'CC(C)(C)OOC(C)(C)CCC(C)(C)OOC(C)(C)C'
 'OC[C@H](O)[C@@H](O)[C@H](O)CO'
 'CCCCCCCC(=O)[O-].CCCCCCCC(=O)[O-].[Zn+2]' 'CC(C)COC(=O)C(C)C']


Next we define our tokenizer and `map` it onto all our data to convert the SMILES strings into lists of tokens. The `BasicSmilesTokenizer` breaks down SMILES roughly at atom level.

In [58]:
tokenizer = dc.feat.smiles_tokenizer.BasicSmilesTokenizer()

train_tok = list(map(tokenizer.tokenize, train_smiles))
valid_tok = list(map(tokenizer.tokenize, valid_smiles))
test_tok = list(map(tokenizer.tokenize, test_smiles))
print(train_tok[0])
len(train_tok)

['C', 'C', '(', 'O', ')', '(', 'P', '(', '=', 'O', ')', '(', 'O', ')', 'O', ')', 'P', '(', '=', 'O', ')', '(', 'O', ')', 'O']


6258

Now we have tokenized versions of all SMILES strings in our dataset. To convert those into lists of integer values we first need to create a list of all possible tokens in our dataset. That list is called the vocabulary. We also add the empty string `""` to our vocabulary in order to correctly handle trailing zeros when decoding zero-padded numericalized SMILES.

In [59]:
flatten = lambda l: [item for items in l for item in items]

all_toks = flatten(train_tok) + flatten(valid_tok) + flatten(test_tok)
vocab = sorted(set(all_toks + [""]))
print(vocab[:12], "...", vocab[-12:])
len(vocab)

['', '#', '(', ')', '-', '.', '/', '1', '2', '3', '4', '5'] ... ['[n+]', '[n-]', '[nH+]', '[nH]', '[o+]', '[s+]', '[se]', '\\', 'c', 'n', 'o', 's']


127

To numericalize tokenized SMILES strings we create a `str2int` dictionary which assigns a number to each token in the dictionary. We also create the reverse `int2str` dictionary and define the corresponding `encode` and `decode` functions. Finally we `map` the `encode` function on the tokenized data to obtain numericalized SMILES data.

In [60]:
str2int = {s:i for i, s in enumerate(vocab)}
int2str = {i:s for i, s in enumerate(vocab)}
print(f"str2int: {dict(list(str2int.items())[:5])} ...")
print(f"int2str: {dict(list(int2str.items())[:5])} ...")

str2int: {'': 0, '#': 1, '(': 2, ')': 3, '-': 4} ...
int2str: {0: '', 1: '#', 2: '(', 3: ')', 4: '-'} ...


In [61]:
encode = lambda s: [str2int[tok] for tok in s]
decode = lambda i: [int2str[num] for num in i]
print(train_smiles[0])
print(encode(train_tok[0]))
print("".join(decode(encode(train_tok[0]))))

CC(O)(P(=O)(O)O)P(=O)(O)O
[19, 19, 2, 24, 3, 2, 25, 2, 16, 24, 3, 2, 24, 3, 24, 3, 25, 2, 16, 24, 3, 2, 24, 3, 24]
CC(O)(P(=O)(O)O)P(=O)(O)O


In [62]:
train_num = list(map(encode, train_tok))
valid_num = list(map(encode, valid_tok))
test_num = list(map(encode, test_tok))
print(train_num[0])

[19, 19, 2, 24, 3, 2, 25, 2, 16, 24, 3, 2, 24, 3, 24, 3, 25, 2, 16, 24, 3, 2, 24, 3, 24]


Lastly, we would like to combine all molecules in a dataset in an `np.array` so they can be served to a model in batches. To achieve that, all sequences have to be of the same length. As in the CoulombMatrix section, we achieve that by appending zeros up to a fixed value.

In [63]:
max_len = max(map(len, train_num + valid_num + test_num))
max_len

240

The longest sequence across all Tox21 datasets has length `240`, so we use that as our fixed length. We create a `zero_pad` function,  `map` it to all numericalized SMILES, and turn them into `np.array`s.

In [64]:
zero_pad = lambda x: x + [0] * (max_len - len(x))

train_numpad = np.array(list(map(zero_pad, train_num)))
valid_numpad = np.array(list(map(zero_pad, valid_num)))
test_numpad = np.array(list(map(zero_pad, test_num)))
train_numpad

array([[19, 19,  2, ...,  0,  0,  0],
       [19, 19,  2, ...,  0,  0,  0],
       [24, 19, 41, ...,  0,  0,  0],
       ...,
       [19, 19,  2, ...,  0,  0,  0],
       [19, 19,  2, ...,  0,  0,  0],
       [23,  1, 19, ...,  0,  0,  0]])

We can check that the zero-padded data still converts back to the correct SMILES string using the `decode` function on a random datapoint.

In [65]:
idx = np.random.randint(0, train_numpad.shape[0], 1).item()
print(train_smiles[idx])
print("".join(decode(train_numpad[idx])))

C=C(C)C(=O)OC
C=C(C)C(=O)OC


The padded data passes the test. It is now in the correct format to be used for training of a sequence model, but it doesn't yet interface nicely with DeepChem's training framework. To change that, we define a `tokenize_smiles` function that combines all the steps spelled out above to process a single datapoint. Additionally, we define a `SmilesFeaturizer` that uses our custom `tokenize_smiles` function in its `_featurize` method and instanciate it as `smiles_featurizer` passing it our `vocab` and `max_len`.

In [66]:
def tokenize_smiles(x, vocab, max_len):
    tokenizer = dc.feat.smiles_tokenizer.BasicSmilesTokenizer()
    str2int = {s:i for i, s in enumerate(vocab)}
    encode = lambda s: [str2int.get(tok, str2int[""]) for tok in s]
    # zero_pad = lambda x: x + [0] * (max_len - len(x))
    # x = tokenizer.tokenize(x)
    # x = encode(x)
    # x = zero_pad(x)
    # return np.array(x)
    try:
        tokens = tokenizer.tokenize(x)
        encoded = encode(tokens[:max_len])  # clip to max_len
        padded = encoded + [0] * (max_len - len(encoded))  # pad to max_len
        return np.array(padded)
    except:
        return np.array([])  # return empty array on failure

class SmilesFeaturizer(dc.feat.Featurizer):
    def __init__(self, feat_func, vocab, max_len):
        self.feat_func = feat_func
        self.vocab = vocab
        self.max_len = max_len

    def _featurize(self, x):
        return self.feat_func(x, self.vocab, self.max_len)

smiles_featurizer = SmilesFeaturizer(tokenize_smiles, vocab, max_len)

Finally, we use the `smiles_featurizer` to create new Tox21 datasets that contain tokenized and numericalized SMILES in their `X` attribute.

In [67]:
tasks, datasets, transformers = dc.molnet.load_tox21(featurizer=smiles_featurizer)
print(datasets[0].X)

[16:11:53] WARNING: not removing hydrogen atom without neighbors
[16:11:53] Explicit valence for atom # 8 Al, 6, is greater than permitted
[16:11:54] Explicit valence for atom # 3 Al, 6, is greater than permitted
[16:11:54] Explicit valence for atom # 4 Al, 6, is greater than permitted
[16:11:54] Explicit valence for atom # 4 Al, 6, is greater than permitted
[16:11:55] Explicit valence for atom # 9 Al, 6, is greater than permitted
[16:11:55] Explicit valence for atom # 5 Al, 6, is greater than permitted
[16:11:55] Explicit valence for atom # 16 Al, 6, is greater than permitted
[16:11:55] Explicit valence for atom # 20 Al, 6, is greater than permitted


[[ 19  19   2 ...   0   0   0]
 [ 19  19   2 ...   0   0   0]
 [ 24  19  41 ...   0   0   0]
 ...
 [ 19  19  19 ...   0   0   0]
 [ 19  39   2 ...   0   0   0]
 [ 19 123   7 ...   0   0   0]]


The datasets are now ready to be used with your custom DeepChem sequence model. Don't forget to wrap your model into the appropriate DeepChem model class.

# Congratulations! Time to join the Community!

Congratulations on completing this tutorial notebook! If you enjoyed working through the tutorial, and want to continue working with DeepChem, we encourage you to finish the rest of the tutorials in this series. You can also help the DeepChem community in the following ways:

## Star DeepChem on [GitHub](https://github.com/deepchem/deepchem)
This helps build awareness of the DeepChem project and the tools for open source drug discovery that we're trying to build.

## Join the DeepChem Gitter
The DeepChem [Gitter](https://gitter.im/deepchem/Lobby) hosts a number of scientists, developers, and enthusiasts interested in deep learning for the life sciences. Join the conversation!

## Citing This Tutorial
If you found this tutorial useful please consider citing it using the provided BibTeX.

In [ ]:
@manual{Intro7,
 title={Going Deeper on Molecular Featurizations},
 organization={DeepChem},
 author={Ramsundar, Bharath},
 howpublished = {\url{https://github.com/deepchem/deepchem/blob/master/examples/tutorials/Going_Deeper_on_Molecular_Featurizations.ipynb}},
 year={2021},
}